In [5]:
! /opt/spark/bin/spark-submit --help

Usage: spark-submit [options] <app jar | python file | R file> [app arguments]
Usage: spark-submit --kill [submission ID] --master [spark://...]
Usage: spark-submit --status [submission ID] --master [spark://...]
Usage: spark-submit run-example [options] example-class [example args]

Options:
  --master MASTER_URL         spark://host:port, mesos://host:port, yarn,
                              k8s://https://host:port, or local (Default: local[*]).
  --deploy-mode DEPLOY_MODE   Whether to launch the driver program locally ("client") or
                              on one of the worker machines inside the cluster ("cluster")
                              (Default: client).
  --class CLASS_NAME          Your application's main class (for Java / Scala apps).
  --name NAME                 A name of your application.
  --jars JARS                 Comma-separated list of jars to include on the driver
                              and executor classpaths.
  --packages                  Comma-

In [7]:
import os
import s3fs
endpoint = "https://"+os.environ['AWS_S3_ENDPOINT']
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': endpoint})

event_log_path="pengfei/spark-history"


fs.touch('s3://'+event_log_path+'/.keep')
fs.info('pengfei/pengfei_test')




{'Key': 'pengfei/pengfei_test',
 'name': 'pengfei/pengfei_test',
 'type': 'directory',
 'Size': 0,
 'size': 0,
 'StorageClass': 'DIRECTORY'}

In [8]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

conf = SparkConf()
conf.set("spark.kubernetes.container.image", "inseefrlab/jupyter-datascience:master") 
conf.set("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT']) 
conf.set("spark.executor.instances", "5")
conf.set("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE']) 
# save event log, can be ommitted
conf.set("spark.eventLog.enabled","true")
conf.set("spark.eventLog.dir","s3a://"+event_log_path)
spark = SparkSession.builder.master("k8s://https://kubernetes.default.svc:443").appName("Python Spark SQL basic example").config(conf).getOrCreate()



In [12]:
! kubectl get pods

NAME                                                   READY   STATUS        RESTARTS   AGE
deleting-pods-with-completed-status-1614949200-5hz5c   0/1     Completed     0          2m23s
jupyter-1614586885-846767d94b-r5zx5                    1/1     Running       0          4d4h
jupyter-1614938651-6cdc6ff6d5-494jt                    1/1     Running       0          178m
pyspark-shell-ac03c9780279915c-exec-5                  0/1     Terminating   0          2m6s
pyspark-shell-ced9457801ec11a7-exec-1                  1/1     Running       0          156m
pyspark-shell-ced9457801ec11a7-exec-2                  1/1     Running       0          156m
pyspark-shell-ced9457801ec11a7-exec-3                  1/1     Running       0          156m
pyspark-shell-ced9457801ec11a7-exec-4                  1/1     Running       0          156m
pyspark-shell-ced9457801ec11a7-exec-5                  1/1     Running       0          156m
ubuntu-1612965548-79c9567b44-nhs9p                     1/1     Running

In [10]:
from pyspark.sql import SparkSession
data_path="s3a://pengfei/sspcloud-demo/data_format/Fire_Department_Calls_for_Service.csv"


AttributeError: 'function' object has no attribute '_get_object_id'

In [11]:
#sc.stop()
spark.stop

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType

fireSchema = StructType([StructField('CallNumber', IntegerType(), True),
                     StructField('UnitID', StringType(), True),
                     StructField('IncidentNumber', IntegerType(), True),
                     StructField('CallType', StringType(), True),                  
                     StructField('CallDate', StringType(), True),       
                     StructField('WatchDate', StringType(), True),       
                     StructField('ReceivedDtTm', StringType(), True),       
                     StructField('EntryDtTm', StringType(), True),       
                     StructField('DispatchDtTm', StringType(), True),       
                     StructField('ResponseDtTm', StringType(), True),       
                     StructField('OnSceneDtTm', StringType(), True),       
                     StructField('TransportDtTm', StringType(), True),                  
                     StructField('HospitalDtTm', StringType(), True),       
                     StructField('CallFinalDisposition', StringType(), True),       
                     StructField('AvailableDtTm', StringType(), True),       
                     StructField('Address', StringType(), True),       
                     StructField('City', StringType(), True),       
                     StructField('ZipcodeofIncident', IntegerType(), True),       
                     StructField('Battalion', StringType(), True),                 
                     StructField('StationArea', StringType(), True),       
                     StructField('Box', StringType(), True),       
                     StructField('OriginalPriority', StringType(), True),       
                     StructField('Priority', StringType(), True),       
                     StructField('FinalPriority', IntegerType(), True),       
                     StructField('ALSUnit', BooleanType(), True),       
                     StructField('CallTypeGroup', StringType(), True),
                     StructField('NumberofAlarms', IntegerType(), True),
                     StructField('UnitType', StringType(), True),
                     StructField('Unitsequenceincalldispatch', IntegerType(), True),
                     StructField('FirePreventionDistrict', StringType(), True),
                     StructField('SupervisorDistrict', StringType(), True),
                     StructField('NeighborhoodDistrict', StringType(), True),
                     StructField('Location', StringType(), True),
                     StructField('RowID', StringType(), True)])
raw_data=